In [2]:
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.ndimage
import spectral_cube
import sys as os
import SpectralCubeTools

In [3]:
COFits = fits.open("./DataFiles/M82-NOEMA-mask.clean.image.fits")
head = COFits[0].header
print(head)
imdata = COFits[0].data
bmajD = head["BMAJ"]
bminD = head["BMIN"] #degrees?
bmaj = bmajD * np.pi/180
bmin = bminD * np.pi/180 #Convert to radians? The header literally does not have the unit :/
beam_in_sr = np.pi*(bmaj/2.0*bmin/2.0)/np.log(2)
rFreq = head["RESTFRQ"]
wcs = WCS(head)
c = 2.99792458e10
kb = 1.380658e-16
print(wcs)
print(np.nanmedian(imdata))

SIMPLE  =                    T /Standard FITS                                   BITPIX  =                  -32 /Floating point (32 bit)                         NAXIS   =                    3                                                  NAXIS1  =                 1400                                                  NAXIS2  =                  900                                                  NAXIS3  =                  160                                                  EXTEND  =                    T                                                  BSCALE  =   1.000000000000E+00 /PHYSICAL = PIXEL*BSCALE + BZERO                 BZERO   =   0.000000000000E+00                                                  BMAJ    =   5.783461938585E-04                                                  BMIN    =   4.594276793522E-04                                                  BPA     =   5.116065467813E+01                                                  BTYPE   = 'Intensity'                   

This cell does all the moment 2 stuff. VERY INTENSE CALCULATION! Do not run this on a weak computer!

In [7]:
from astropy.utils import data

from SpectralCubeTools.spectral_cube_tools import characterize

testCube = spectral_cube.SpectralCube.read("./DataFiles/M82-NOEMA-mask.clean.image.fits")
testCube.allow_huge_operations = True #Heed the warning, the next line will fry your computer.
noise = characterize.calc_noise_in_cube(testCube) 


SpectralCube with shape=(160, 900, 1400) and unit=Jy / beam:
 n_x:   1400  type_x: RA---ARC  unit_x: deg    range:   148.744223 deg:  149.193015 deg
 n_y:    900  type_y: DEC--ARC  unit_y: deg    range:    69.635108 deg:   69.734996 deg
 n_s:    160  type_s: VRAD      unit_s: m / s  range:  -185000.000 m / s:  610000.000 m / s


c:\Users\Colton\AppData\Local\Programs\Python\Python312\Lib\site-packages\astropy\stats\funcs.py:815: RuntimeWarning: All-NaN slice encountered
  data_median = func(data, axis=axis)
c:\Users\Colton\AppData\Local\Programs\Python\Python312\Lib\site-packages\astropy\stats\funcs.py:821: RuntimeWarning: All-NaN slice encountered
  result = func(np.abs(data - data_median), axis=axis, overwrite_input=True)
c:\Users\Colton\AppData\Local\Programs\Python\Python312\Lib\site-packages\astropy\stats\funcs.py:815: RuntimeWarning: All-NaN slice encountered
  data_median = func(data, axis=axis)
c:\Users\Colton\AppData\Local\Programs\Python\Python312\Lib\site-packages\astropy\stats\funcs.py:821: RuntimeWarning: All-NaN slice encountered
  result = func(np.abs(data - data_median), axis=axis, overwrite_input=True)
c:\Users\Colton\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\ndimage\_filters.py:1850: RuntimeWarning: All-NaN slice encountered
  _nd_image.generic_filter(input, function, fo

In [10]:
from SpectralCubeTools.spectral_cube_tools import identify_signal

maskedCube = identify_signal.find_signal_in_cube(testCube, noise, snr_hi=5) #not nearly as intense as the noise calculation, but its still intensive

In [11]:
m2 = maskedCube.with_spectral_unit(u.km/u.s).moment2() #Creates moment 2 (variance) cube in km^2/s^2
m2 = m2**(1/2) #dispersion = sqrt(variance)
hdu = m2.hdu
hdu.writeto("./DataFiles/M82_moment2.fits", overwrite=True) #Save it

c:\Users\Colton\AppData\Local\Programs\Python\Python312\Lib\site-packages\spectral_cube\_moments.py:87: RuntimeWarning: invalid value encountered in divide
  return result / weights


km / s


c:\Users\Colton\AppData\Local\Programs\Python\Python312\Lib\site-packages\astropy\units\quantity.py:671: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


The next cells create a moment 0 map

In [6]:
slices = [[0,0,i] for i in range(160)] 
pixcord = np.array(slices)
world = wcs.wcs_pix2world(pixcord, 1) #Third element is radial velocity
rVels = world[:,2]
freq = np.mean([rFreq*(1-i/c) for i in rVels]) #Calculate frequencies to use in unit conversion
print(freq)


115270405280.55762


In [7]:

convFactor = c**2/beam_in_sr/1e23/(2*kb*freq**2) #Textbook formula
imdata *= convFactor #Converted from Jansky/beam to K 

26.709481799729076


In [11]:
#Create spectral cube object. We do it now because it is easier to convert the unit when the cube is a numpy array.
cube = spectral_cube.SpectralCube(data = imdata, wcs=wcs) 

In [12]:

m0 = cube.moment(order=0) #Calculates moment 0 map in K m/s
m0 = m0/1000 #convert m/s to km/s
hdu = m0.hdu
hdu.header["BUNIT"] = "K km s^-1" #We did manual unit conversion so we need to update the header.
hdu.writeto("./DataFiles/M82_moment0.fits", overwrite=True) #Save the map

K km s^-1
